# Initial Setting
I started off trying to translate the R code to Python, however there were problems trying to formate the data.
I used R Studio to run the R code again. This code worked, however due to file size the code only ran until line 35, from here the data was too large to process.
Therefore I loaded the CSV that R had outputted and continued using Python.

Steps:
 1. sample 10/100K rows.
 2. split : meta & drug db.
 3. define transform from R code.
 4. apply transforms.
 5. train model.

#### 1. Load Data

In [1]:
import pandas as pd

In [82]:
pres = pd.read_csv(r"P:/data_science/prescriber-info.csv", header=0, sep=",", nrows = 10000000)

In [3]:
meta = pd.read_csv(r"D:/NPI/PartD_Prescriber_PUF_NPI_17.txt", header=0, sep="\t", nrows =1000000)

In [12]:
final_data.to_csv(r"P:/data_science/r_check.csv")

In [6]:
pres.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_city', 'nppes_provider_state', 'specialty_description',
       'description_flag', 'drug_name', 'generic_name', 'bene_count',
       'total_claim_count', 'total_30_day_fill_count', 'total_day_supply',
       'total_drug_cost', 'bene_count_ge65', 'bene_count_ge65_suppress_flag',
       'total_claim_count_ge65', 'ge65_suppress_flag',
       'total_30_day_fill_count_ge65', 'total_day_supply_ge65',
       'total_drug_cost_ge65'],
      dtype='object')

In [83]:
pres

,npi,Gender,State,Credentials,Specialty,Opioid.Prescriber
0,1043380199,M,CA,0,Family Practice,1
1,1164488268,M,OH,M.D.,Internal Medicine,1
2,1144208828,M,MN,M.D.,Gynecological Oncology,0
3,1093876146,M,TX,D.D.S.,Dentist,1
4,1093754574,M,MO,MD,Emergency Medicine,1
...,...,...,...,...,...,...
24995,1053375386,M,TN,MD,Family Practice,1
24996,1104898386,M,CA,M.D.,Internal Medicine,1
24997,1194082602,F,VA,0,Family Practice,1
24998,1144632506,M,NY,D.O.,Family Practice,0


In [8]:
meta.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip5', 'nppes_provider_zip4',
       'nppes_provider_state', 'nppes_provider_country',
       'specialty_description', 'description_flag',
       'medicare_prvdr_enroll_status', 'total_claim_count',
       'total_30_day_fill_count', 'total_drug_cost', 'total_day_supply',
       'bene_count', 'ge65_suppress_flag', 'total_claim_count_ge65',
       'total_30_day_fill_count_ge65', 'total_drug_cost_ge65',
       'total_day_supply_ge65', 'bene_count_ge65_suppress_flag',
       'bene_count_ge65', 'brand_suppress_flag', 'brand_claim_count',
       'brand_drug_cost', 'generic_suppress_flag', 'generic_claim_count',
       'generic_drug_cost', 'other_suppress_flag', 'other_claim_count',
       'other_drug_cost', 'mapd_suppress

In [5]:
meta

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,92.0,220.0,14.0,0.0,0.0,0.0,0.0,143.0,91.0,2.1685
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,92.0,195.0,58.0,NaN,NaN,0.0,NaN,143.0,133.0,1.8029
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,...,17.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0598
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,4.5148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1861403412,BOSS,EUGENE,F,MD,M,I,3400 MAIN ST,SUITE 6,SPRINGFIELD,...,242.0,520.0,38.0,0.0,39.0,NaN,NaN,453.0,158.0,1.1914
999996,1861403438,SOPKOWICZ,LINDA,S,DDS,F,I,1201 OAK STREET,NaN,WEST BEND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
999997,1861403578,KADLEC,SUSAN,NaN,NaN,F,I,1700 MYRTLE AVE,NaN,PLAINFIELD,...,0.0,56.0,NaN,NaN,NaN,0.0,0.0,66.0,12.0,0.8984
999998,1861403628,RODRIGUES,CELSO,F,M.D.,M,I,2674 FOX RUN DR,NaN,IMPERIAL,...,23.0,30.0,NaN,0.0,NaN,0.0,NaN,15.0,32.0,2.3517


## Pivot and Format R data w/ meta


In [84]:
pivot_data = pres.pivot(index='npi',columns='drug_name', values='total_claim_count')

KeyError: 'drug_name'

In [7]:
pivot_data

drug_name,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,ALENDRONATE.SODIUM,ALLOPURINOL,ALPHAGAN.P,ALPRAZOLAM,AMIODARONE.HCL,AMITIZA,AMITRIPTYLINE.HCL,...,VENTOLIN.HFA,VERAPAMIL.ER,VESICARE,VICTOZA.3.PAK,VOLTAREN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE
npi,,,,,,,,,,,,,,,,,,,,,
1003000126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN,NaN,NaN
1003000142,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003000167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003000282,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003000407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205084662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1205084688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1205084704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge Prescriber info with Meta-Data

In [8]:
format_data = meta#[['npi','nppes_provider_gender', 'nppes_provider_state', 'nppes_credentials', 'specialty_description']]

In [9]:
merge_data = format_data.merge(pivot_data, left_on='npi', right_on="npi")

In [29]:
final_data = merge_data.fillna(0)

In [30]:
final_data

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,VENTOLIN.HFA,VERAPAMIL.ER,VESICARE,VICTOZA.3.PAK,VOLTAREN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE
0,1003000126,ENKESHAFI,ARDALAN,0,M.D.,M,I,900 SETON DR,0,CUMBERLAND,...,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0
1,1003000142,KHALIL,RASHID,0,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,0,DAYTON,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1003000407,GIRARDI,DAVID,J,D.O.,M,I,100 HOSPITAL RD,0,BROOKVILLE,...,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181823,1205084662,TUMINELLI,FRANK,J,DMD,M,I,23 BOND ST,0,GREAT NECK,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181824,1205084688,IKBAL,KATRINY,A,D.O.,F,I,12005 BEE CAVE RD,SUITE A-1,AUSTIN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181825,1205084704,GRIFFIN,TARA,M,DMD,F,I,3135 THOMAS DR,0,PANAMA CITY BEACH,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181826,1205084894,OLIVEIRA,CHAD,D,DDS,M,I,1215 N IRWIN ST,0,HANFORD,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Seperate Files

In [20]:
meta_data = final_data[['npi','nppes_provider_state','nppes_credentials']]

In [21]:
meta_data

,npi,nppes_provider_state,nppes_credentials
0,1003000126,MD,M.D.
1,1003000142,OH,M.D.
2,1003000167,NV,DDS
3,1003000282,TN,FNP
4,1003000407,PA,D.O.
...,...,...,...
181823,1205084662,NY,DMD
181824,1205084688,TX,D.O.
181825,1205084704,FL,DMD
181826,1205084894,CA,DDS


In [22]:
add_on = meta[['npi','nppes_provider_last_org_name', 'nppes_provider_first_name','nppes_provider_gender' ,'specialty_description','nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip5', 'nppes_provider_zip4', 'nppes_provider_country',]]

In [23]:
meta_add_on = format_data.merge(pivot_data, left_on='npi', right_on="npi")

In [24]:
meta_add_on

,npi,nppes_provider_state,nppes_credentials,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_gender,specialty_description,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip5,nppes_provider_zip4,nppes_provider_country
0,1003000126,MD,M.D.,ENKESHAFI,ARDALAN,M,Internal Medicine,900 SETON DR,NaN,CUMBERLAND,21502.0,1854.0,US
1,1003000142,OH,M.D.,KHALIL,RASHID,M,Anesthesiology,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,43623.0,3536.0,US
2,1003000167,NV,DDS,ESCOBAR,JULIO,M,Dentist,5 PINE CONE RD,NaN,DAYTON,89403.0,7482.0,US
3,1003000282,TN,FNP,BLAKEMORE,ROSIE,F,Nurse Practitioner,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,37243.0,1.0,US
4,1003000407,PA,D.O.,GIRARDI,DAVID,M,Family Practice,100 HOSPITAL RD,NaN,BROOKVILLE,15825.0,1367.0,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...
181823,1205084662,NY,DMD,TUMINELLI,FRANK,M,Dentist,23 BOND ST,NaN,GREAT NECK,11021.0,2025.0,US
181824,1205084688,TX,D.O.,IKBAL,KATRINY,F,Family Practice,12005 BEE CAVE RD,SUITE A-1,AUSTIN,78738.0,NaN,US
181825,1205084704,FL,DMD,GRIFFIN,TARA,F,Dentist,3135 THOMAS DR,NaN,PANAMA CITY BEACH,32408.0,6256.0,US
181826,1205084894,CA,DDS,OLIVEIRA,CHAD,M,Dentist,1215 N IRWIN ST,NaN,HANFORD,93230.0,2929.0,US


In [25]:
meta_add_on.to_csv(r'P:/data_science/github/Medication_Prediction/Data/SortedData/NPI_Meta.csv')

In [26]:
New_final = final_data.drop(['nppes_provider_gender', 'nppes_provider_state', 'nppes_credentials', 'specialty_description'], 1)

In [28]:
New_final.to_csv(r'P:/data_science/github/Medication_Prediction/Data/SortedData/NPI_Drug_exclusive.csv')

## Save Files

In [86]:
final_data.to_csv(r"P:/data_science/github/Medication_Prediction/SortedData/Prescriber-Info-LargeData.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'P:/data_science/github/Medication_Prediction/SortedData/Prescriber-Info-LargeData.csv'

In [48]:
opioid_prescriber ={'opioid_prescriber':[]}

In [87]:
sorted_data= pd.read_csv(r"P:/data_science/prescriber-info-count.csv", header=0, sep=",")

In [88]:
sorted_data

,npi,Gender,State,Credentials,Specialty,Opioid.Prescriber
0,1104897172,F,PA,DO,Family Practice,0
1,1164701173,F,IN,APRN,Nurse Practitioner,0
2,1164736930,F,MO,M.D.,Nephrology,0
3,1184615296,F,WI,PA-C,Physician Assistant,0
4,1124260096,M,GA,D.M.D.,Dentist,0
...,...,...,...,...,...,...
181823,1023098696,F,MT,D.O.,Family Practice,1
181824,1013972025,M,NC,M.D.,Psychiatry,0
181825,1184663346,F,TX,MD,Emergency Medicine,1
181826,1154323749,F,TN,MD,Dermatology,0


In [89]:
meta_add_on = sorted_data.merge(pivot_data, left_on='npi', right_on="npi")

In [90]:
Final_model_sort=meta_add_on.fillna(0)

In [91]:
Final_model_sort

,npi,Gender,State,Credentials,Specialty,Opioid.Prescriber,ACETAMINOPHEN.CODEINE,ACYCLOVIR,ADVAIR.DISKUS,ALENDRONATE.SODIUM,...,VENTOLIN.HFA,VERAPAMIL.ER,VESICARE,VICTOZA.3.PAK,VOLTAREN,WARFARIN.SODIUM,XARELTO,ZETIA,ZIPRASIDONE.HCL,ZOLPIDEM.TARTRATE
0,1104897172,F,PA,DO,Family Practice,0,0.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
1,1164701173,F,IN,APRN,Nurse Practitioner,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1164736930,F,MO,M.D.,Nephrology,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1184615296,F,WI,PA-C,Physician Assistant,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1124260096,M,GA,D.M.D.,Dentist,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181823,1023098696,F,MT,D.O.,Family Practice,1,25.0,0.0,43.0,33.0,...,22.0,0.0,0.0,0.0,0.0,17.0,17.0,0.0,0.0,17.0
181824,1013972025,M,NC,M.D.,Psychiatry,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181825,1184663346,F,TX,MD,Emergency Medicine,1,22.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
181826,1154323749,F,TN,MD,Dermatology,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
Final_model_sort.to_csv(r"P:/data_science/github/Medication_Prediction/Data/SortedData/Prescriber-info-count.csv")

# Try run model